In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import glob
from pathlib import Path
import seaborn as sns
import os
from experiment_utils import Experiment, ExperimentFilter, Plotting

In [ ]:
experiments = Experiment.get_experiments()
experiments = list(filter(ExperimentFilter.by_experiment_name('avx-frequency-license-bands-without-isst'), experiments))
experiment = ExperimentFilter.get_latest(experiments)
experiment

In [ ]:
variants = glob.glob(f'{experiment.path}/*/*/*')
variants

In [ ]:
def read_firestarter_from_lic(folder: str) -> pd.DataFrame:
    """
    folder: The folder that contains the measurements for the different number of cores.
    """
    df = pd.DataFrame()

    for file in glob.glob(f'{folder}/firestarter-*'):
        _name, index = Path(file).stem.split('-')
        index = int(index) + 1

        read_df = pd.read_csv(file)

        for _index, row in read_df.iterrows():
            metric = row['metric']
            value = row['average']
            df = pd.concat([df, pd.DataFrame([{'metric': metric, 'value': value, 'core_count': index}])])

    return df

In [ ]:
def read_uncore_from_lic(folder: str) -> pd.DataFrame:
    """
    folder: The folder that contains the measurements for the different number of cores.
    """
    df = pd.DataFrame()

    for file in glob.glob(f'{folder}/uncore-freq-*'):
        _name, _name2, index = Path(file).stem.split('-')
        index = int(index) + 1

        read_df = pd.read_csv(file)

        for _index, row in read_df.iterrows():
            metric = row['name']
            value = row['average']
            df = pd.concat([df, pd.DataFrame([{'metric': metric, 'value': value, 'core_count': index}])])

    return df

In [ ]:
data = pd.DataFrame()

for variant in variants:
    core_frequency, uncore_frequency, license = variant.split('/')[-3:]

    lic_data = read_firestarter_from_lic(variant)
    lic_data['core_frequency'] = core_frequency
    lic_data['uncore_frequency'] = uncore_frequency
    lic_data['license'] = license

    data = pd.concat([data, lic_data])

    lic_data = read_uncore_from_lic(variant)
    lic_data['core_frequency'] = core_frequency
    lic_data['uncore_frequency'] = uncore_frequency
    lic_data['license'] = license
    data = pd.concat([data, lic_data])

data

In [ ]:
working_data = data
working_data = working_data.pivot_table(index=['license', 'core_count', 'core_frequency', 'uncore_frequency'], values='value', columns=['metric'])
working_data['freq_times_power'] = working_data['sysfs-powercap-rapl/package-0'] / working_data['perf-freq']
working_data = working_data.reset_index()

license_level_to_name = {
    'lic0': 'SSE',
    'lic1': 'AVX2',
    'lic2': 'AVX512',
    'lic3': 'AMX'
}

LIC_NAME = 'License name'
CORE_COUNT_NAME = 'Number of active cores'
LICENSE_NAME = 'Used cdyn class'
POWER_WATT_NAME = 'Power draw of package 0 [W]'
CORE_FREQUENCY_NAME = 'Average frequency of running cores [GHz]'
UNCORE_FREQUENCY_NAME = 'Average uncore frequency [GHz]'


CORE_COUNT = 'Number of active cores'
MAX_TURBO_FREQ = 'Maximum opportunistic turbo frequency [MHz]'
POWER_WATT = 'Power draw of package 0 [W]'


working_data[LIC_NAME] = working_data['license'].apply(lambda v: license_level_to_name[v])
working_data[CORE_COUNT_NAME] = working_data['core_count']
working_data[LICENSE_NAME] = working_data['license']

working_data[POWER_WATT_NAME] = working_data['sysfs-powercap-rapl/package-0']
working_data[CORE_FREQUENCY_NAME] = working_data['perf-freq']
working_data[UNCORE_FREQUENCY_NAME] = working_data['package_00_die_00'] / 1e6

working_data

In [ ]:
hue_to_color = {
    "AMX": sns.color_palette()[0],
    "AVX512": sns.color_palette()[1],
    "AVX2": sns.color_palette()[2],
    "SSE": sns.color_palette()[3]
}
hue_order=list(hue_to_color.keys())

In [ ]:
g = sns.FacetGrid(working_data, col="core_frequency",  row="uncore_frequency", height=5, row_order=["0x0808", "0x1010", "0x1919", "0x0819"], col_order=["800", "1600", "2000", "performance"])
g.map(sns.scatterplot, CORE_COUNT_NAME, POWER_WATT_NAME, LIC_NAME, linewidth=1.5, hue_order=hue_order)

g.add_legend()

buckets = [28,34,40,44,48,50,54,56]
for axi in g.axes:
    for axj in axi:
        for i in buckets:
            axj.axvline(x=i, color='black', dashes=[2], linewidth=1)
            axj.set_xlim(0, 58)
            axj.set_xticks([0] + buckets)
            axj.tick_params(axis='x', pad=0, rotation=45)

Plotting.savefig(experiment, 'package-power.pdf', annotations_y_offset=0, annotations_y_spacing=0.025)

In [ ]:
g = sns.FacetGrid(working_data, col="core_frequency",  row="uncore_frequency", height=5, row_order=["0x0808", "0x1010", "0x1919", "0x0819"], col_order=["800", "1600", "2000", "performance"])
g.map(sns.scatterplot, CORE_COUNT_NAME, CORE_FREQUENCY_NAME, LIC_NAME, linewidth=1.5, hue_order=hue_order)
g.add_legend()

buckets = [28,34,40,44,48,50,54,56]
for axi in g.axes:
    for axj in axi:
        for i in buckets:
            axj.axvline(x=i, color='black', dashes=[2], linewidth=1)
            axj.set_xlim(0, 58)
            axj.set_xticks([0] + buckets)
            axj.tick_params(axis='x', pad=0, rotation=45)

Plotting.savefig(experiment, 'core-frequency.pdf', annotations_y_offset=0, annotations_y_spacing=0.025)

In [ ]:
g = sns.FacetGrid(working_data, col="core_frequency",  row="uncore_frequency", height=5, row_order=["0x0808", "0x1010", "0x1919", "0x0819"], col_order=["800", "1600", "2000", "performance"])
g.map(sns.scatterplot, CORE_COUNT_NAME, UNCORE_FREQUENCY_NAME, LIC_NAME, linewidth=1.5, hue_order=hue_order)

g.add_legend()

buckets = [28,34,40,44,48,50,54,56]
for axi in g.axes:
    for axj in axi:
        for i in buckets:
            axj.axvline(x=i, color='black', dashes=[2], linewidth=1)
            axj.set_xlim(0, 58)
            axj.set_xticks([0] + buckets)
            axj.tick_params(axis='x', pad=0, rotation=45)

Plotting.savefig(experiment, 'uncore-frequency.pdf', annotations_y_offset=0, annotations_y_spacing=0.025)

In [ ]:
g = sns.FacetGrid(working_data[working_data["core_frequency"] == "performance"], col="uncore_frequency", height=5, col_order=["0x0808", "0x1010", "0x1919", "0x0819"], col_wrap=2)
g.map(sns.scatterplot, CORE_COUNT_NAME, CORE_FREQUENCY_NAME, LIC_NAME, linewidth=1.5, hue_order=hue_order)
g.add_legend()

buckets = [28,34,40,44,48,50,54,56]
for axi in g.axes:
    for i in buckets:
        axi.axvline(x=i, color='black', dashes=[2], linewidth=1)
        axi.set_xlim(0, 58)
        axi.set_xticks([0] + buckets)
        axi.tick_params(axis='x', pad=0, rotation=45)

Plotting.savefig(experiment, 'core-frequency-performance.pdf', annotations_y_offset=0, annotations_y_spacing=0.025)